In [1]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

## Creative file meta

In [2]:
rsna = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\RSNA.csv')
ddsm = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM.csv')
cmmd = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CMMD.csv')
cdd = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\cdd_cesm.csv')


In [3]:
cdd = cdd[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
cmmd = cmmd[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
rsna = rsna[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
ddsm = ddsm[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]

In [4]:
meta = pd.read_csv('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_data.csv')
meta

,Unnamed: 0,patient_id,image_id,view,laterality,cancer
0,0,10006,1874946579,CC,R,0
1,1,10006,1864590858,MLO,R,0
2,2,10006,462822612,CC,L,0
3,3,10006,1459541791,MLO,L,0
4,4,10011,541722628,CC,R,0
...,...,...,...,...,...,...
53745,53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,53746,D2-0748,D2-0748-1-4,CC,R,1
53747,53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,53748,D2-0749,D2-0749-1-2,CC,L,1


In [5]:
from Setting_data import *

In [6]:
sd = Setting_data(meta)
sd.multi_process(list(meta.patient_id.unique()))

d:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Cau_truc_data\Setting_data.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.id_img_zip[i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'


In [7]:
sd.meta_pro

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,D2-0748,D2-0748-1-4,CC,R,1
53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,D2-0749,D2-0749-1-2,CC,L,1


In [4]:
total_len = len(cdd) + len(cmmd) + len(rsna) + len(ddsm)
total_len

59468

In [5]:
meta =pd.concat([cdd, cmmd, rsna, ddsm]).reset_index()
meta = meta[["patient_id",	"image_id"	,"view",	"laterality"	,	"cancer"]]
meta

,patient_id,image_id,view,laterality,cancer
0,1CDD,P1-L-DM-MLO,MLO,L,1
1,1CDD,P1-L-DM-MLO,CC,L,0
2,3CDD,P3-R-DM-CC,CC,R,0
3,3CDD,P3-R-DM-MLO,MLO,R,0
4,3CDD,P3-L-DM-CC,CC,L,1
...,...,...,...,...,...
59463,4607DDSM,5119,MLO,R,0
59464,4608DDSM,5120,CC,L,0
59465,4608DDSM,5121,MLO,L,0
59466,4608DDSM,5122,CC,R,0


In [12]:
meta.to_csv('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main1\meta_data.csv')

## Move data in folder CC, MLO

In [6]:
meta.view.value_counts()

MLO    29734
CC     29734
Name: view, dtype: int64

In [7]:
# par -> view -> lat -> view_lat_id_image

In [ ]:
"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_VIEW\ff5c656fb6757b708d64d2ebdd33c421\CC\L\CC_L_11ae77d09d90780bb56ed52bcefb4272.png"

In [16]:
os.makedirs('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main1\CC')
os.makedirs('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main1\MLO')


In [8]:
from move_cc_mlo import *

In [9]:
m = move(meta)
m.my_method(list(meta.index))

In [10]:
from sklearn.model_selection import train_test_split


In [36]:
class Evaluate:
    def __init__(self, meta_pro, name):
        self.al_data = meta_pro.reset_index()
        self.name = name
    def quatity_cc_mlo(self):
        return f"\nSo luong CC và MLO\n{self.al_data.view.value_counts()}\n"
        # print(self.al_data.view.value_counts())
    def lelf_right(self):
        return f"\nSo luong L và R\n{self.al_data.groupby(['view', 'laterality']).size()}\n"
        # print(self.al_data.groupby(['view', 'laterality']).size())
    def show_one(self, par):
        return self.al_data[self.al_data.patient_id == par]
    def quanlity_cancer(self):
        sl = len(self.al_data[self.al_data.cancer == 1])
        slno = len(self.al_data[self.al_data.cancer == 0])
        return f"So luong img cancer: {sl} \n So luong img no cancer: {slno}"
        # print(self.al_data[self.al_data.cancer == 1])
    def show(self, path):
        ls = len(self.al_data)
        par = len(self.al_data.patient_id.unique())
        # print( 
        # self.quatity_cc_mlo()
        # self.lelf_right()
        # self.quanlity_cancer()
        f = open(path, 'a', encoding='utf-8')
        f.write(f'Dataset {self.name} \n Số lượng image: {ls} \n Số luong patient: {par} {self.quatity_cc_mlo()} {self.lelf_right()} {self.quanlity_cancer()}\n\n +++++++++++++++\n\n')
        f.close()

In [111]:
def save(cancer_train, nocancer_train, cancer_val, nocancer_val, cancer_test, nocancer_test, name = "Rong",  name_folder = "Rong", view = None):
    path = f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main1/Dataset/{name_folder}/{view}/{name}'
    os.makedirs(path)
    # print(len(cancer_train), len(nocancer_train))
    # print(nocancer_train)
    train = pd.concat([cancer_train, nocancer_train]).reset_index().drop('index', axis=1)
    
    val = pd.concat([cancer_val, nocancer_val]).reset_index().drop('index', axis=1)
    test = pd.concat([cancer_test, nocancer_test]).reset_index().drop('index', axis=1)
    # test = pd.read_csv(r'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM.csv', index_col='Unnamed: 0')
    ev = Evaluate(train, "Train")
    ev1 = Evaluate(val, "Val")
    ev2 = Evaluate(test, "Test")
    ev.show(f'{path}/description.txt')
    ev1.show(f'{path}/description.txt')
    ev2.show(f'{path}/description.txt')
    train.to_csv(f'{path}/train.csv')
    val.to_csv(f'{path}/val.csv')
    test.to_csv(f'{path}/test.csv')

In [112]:
class cplit_data:
    def __init__(self, data, name_data, name_view):
        self.data = data
        self.name_data = name_data
        self.name_view = name_view
    def split_cancer(self, cancer):
        # Chia dữ liệu thành tập train và tập còn lại (test + validation)
        X_train, X_test_val = train_test_split(cancer, test_size=0.3, random_state=42)

        # Chia tập còn lại thành tập test và tập validation
        X_test, X_val = train_test_split(X_test_val, test_size=0.33, random_state=42)
        return X_train, X_test, X_val
    def split_rate(self, rate, data, lens_train, len_val, len_test):
        df =  pd.DataFrame({'age': []})
        if lens_train> len(data) and rate == 1:
            return data
        if lens_train> len(data) and rate > 1: return df
        
        total = (len(data))/lens_train
        if total == 0 and rate == 1:
            return data
        
        if rate <= total:
            ls = rate * lens_train
            df_sample = data.sample(n=ls, random_state=42)
            return df_sample
        
        return df
    def process(self):
        cancer = self.data[self.data.cancer == 1]
        cancer_train, cancer_test, cancer_val = self.split_cancer(cancer)
        
        nocancer = self.data[self.data.cancer == 0]
        
        nocancer_test = nocancer.sample(n=len(cancer_test), random_state=42)
        
        remain = nocancer[~nocancer.image_id.isin(nocancer_test.image_id)]
        
        nocancer_val = remain.sample(n=len(cancer_val), random_state=42)
            
        remain = remain[~remain.image_id.isin(nocancer_val.image_id)]
        for i in range(1, 4):
            nocancer_train = self.split_rate(i, remain, len(cancer_train), len(cancer_val), len(cancer_test))
            if len(nocancer_train) !=0:
                # train = pd.concat([cancer_train, nocancer_train])
                # print(type(nocancer_train))
                # return
                save(cancer_train, nocancer_train, cancer_val, nocancer_val, cancer_test, nocancer_test, name = f"rate1{i}",  name_folder = self.name_data, view = self.name_view)
                # return
            else :
                return
            
            

In [113]:
# sp_rsna_cc = cplit_data(rsna[rsna.view == 'CC'], 'RSNA', 'CC')
# sp_rsna_cc.process()

In [114]:
# sp_rsna_MLO = cplit_data(rsna[rsna.view == 'MLO'], 'RSNA', 'MLO')
# sp_rsna_MLO.process()

In [115]:
sp_DDSM_CC = cplit_data(ddsm[ddsm.view == 'CC'], 'DDSM', 'CC')
sp_DDSM_CC.process()

In [116]:
sp_DDSM_MLO = cplit_data(ddsm[ddsm.view == 'MLO'], 'DDSM', 'MLO')
sp_DDSM_MLO.process()

In [117]:
sp_CMMD_CC = cplit_data(cmmd[cmmd.view == 'CC'], 'CMMD', 'CC')
sp_CMMD_CC.process()


In [118]:
sp_CMMD_MLO = cplit_data(cmmd[cmmd.view == 'MLO'], 'CMMD', 'MLO')
sp_CMMD_MLO.process()

In [119]:
sp_CDD_CC = cplit_data(cdd[cdd.view == 'CC'], 'CDD', 'CC')
sp_CDD_CC.process()
sp_CDD_MLO = cplit_data(cdd[cdd.view == 'MLO'], 'CDD', 'MLO')
sp_CDD_MLO.process()